In [1]:
# import all necessary packages
import tensorflow as tf
import os
import numpy as np
import glob
import time

In [2]:
# setup the constants
IMG_SHAPE = 224  # size of our input image needed for our model IMG_SHAPE x IMG_SHAPE x 3 (color)
MODEL_DIRECTORY = 'C:/Users/thoma/Documents/CSU East Bay/2nd Year/Fall 2019/CS 663/Projects/Project 3/Part 1'
DATA_DIRECTORY = 'C:/Users/thoma/Documents/CSU East Bay/2nd Year/Fall 2019/CS 663/Projects/Project 3/Video Dataset (Full)'

# labels for the 3 classes
LABELS = ['MoveLeft','MoveRight','MoveStraight']

DECISION_DIFFERENCE_THRESHOLD = 0.1
SEQUENCE_LENGTH = 40
FEATURE_LENGTH = 1280

In [3]:
# function to load a model from an h5 file
def loadModelFrom_H5_File(model_file):
    new_model = tf.keras.models.load_model(model_file)
    
    return new_model

In [4]:
# function to load and prepare the video feature data from a file
def loadData(data_path, sequence_length, feature_length):
    features = np.load(data_path)
    features = np.expand_dims(features,axis=0)
    padded_sequence = np.zeros((1,sequence_length, feature_length))
    padded_sequence[0:len(features)] = np.array(features)
    
    return padded_sequence

In [5]:
# function to use an LSTM model to make a prediction on live video data
def predict(input, model):
    prediction = model.predict(input, batch_size=1, verbose=0)
    
    return prediction

In [6]:
# function to make a classification decision and return a label
def makeDecision(predictions, class_labels):
    max_index = np.argmax(predictions)
    predictions2 = np.delete(predictions,[max_index],None)
    max_index2 = np.argmax(predictions2)
    
    if(max_index2 >= max_index):
        max_index2 = max_index2 + 1
    
    if(predictions[0][max_index] >= predictions[0][max_index2] and 
      (predictions[0][max_index]-predictions[0][max_index2]) > DECISION_DIFFERENCE_THRESHOLD):
        label = class_labels[max_index]
    else:
        label = "Unknown"
    
    return label

In [7]:
# load the saved LSTM model
model_files = os.path.join(MODEL_DIRECTORY, '*.h5')
model_paths = tf.io.gfile.glob(model_files)
model_file = model_paths[0]
model = loadModelFrom_H5_File(model_file)   

In [8]:
# get all of the filenames and paths for the testing dataset from the
# .txt file that was created from the FeatureExtractionNotebook
testing_file = os.path.join(DATA_DIRECTORY,'TestList.txt')

with open(testing_file) as f:
    testing_list = [row.strip() for row in list(f)]

In [9]:
# make a folder to store all of the batch prediction results if it doesn't already exist
results_directory = os.path.join(DATA_DIRECTORY,'BatchTestingPredictionResults')
if not os.path.exists(results_directory):
    os.mkdir(results_directory)

In [10]:
# create a new .txt file to save the results of the batch prediction
timestr = time.strftime("%Y%m%d-%H%M%S")
batch_prediction_filename = "BatchPredictions_" + timestr + ".txt"
batch_prediction_filename = os.path.join(results_directory,batch_prediction_filename)
prediction_file = open(batch_prediction_filename, 'w+')

In [11]:
# make and save predictions for all of the testing files in testing_list
for i in range(len(testing_list)):
    features = loadData(testing_list[i], SEQUENCE_LENGTH, FEATURE_LENGTH)
    prediction = predict(features, model)
    classification = makeDecision(prediction, LABELS)
    prediction_file.write("Prediction: " + classification + "\n" 
                          "Prediction Data Source: " + testing_list[i] + "\n\n\n")

prediction_file.close()